In this notebook, we're going to work with an extended version of the "Heatlh Care for All" dataset will lots of new columns. We will start reviewing the manual data cleaning process for numerical, and categorical columns for some columns. Later, during the week we will cover automated ways to select the best subsets of columns to be used in our analysis.

# 7.01


# Loading libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)


# Reading data

In [ ]:
data = pd.read_csv('learningSet.csv')

In [ ]:
print(data.shape)
data.head()

Column "TARGET_B" encodes if the customer answered or not the call

In [ ]:
data['TARGET_B'].value_counts()

Column "TARGET_D" provide the donation amount.

In [ ]:
data['TARGET_D'].describe()

In [ ]:
data[data['TARGET_B']>0]['TARGET_D'].describe()

To solve this problem we will first build a classification model to predict who will more likely respond and then for those respondents, we will build a regression model to predict the donation amount.

Then we can use the cost matrix to calculate the total benefit from the donations (remember that the company has to do a mailing champaing and each envelope costs some money).

Some of the challenges with the dataset are as follows:

* Large number of features: The data set has over 450 features. Hence selecting the right features for the model is very critical and at the same time it is not easy as the same traditional ways of removing features is not effective given the large number of features. Apart from feature selection, feature extraction (creating your own features using the existing features) is also not easy in this case.

* Sparsity of the dataset: There are a lot of features with a large number of null values.

* Data imbalance: For developing a classification, there is a huge imbalance in the training dataset with only approximately 5000 values for one category as compared to cover 95,000 instances for the other category.

# Review data cleaning process

There are a lot of columns that have a very high percentage of null values. It is a highly sparse dataset.
We can decide on a threshold and then remove those variables. There is no rule of thumb to decide on this threshold value.
Sometimes it can as low as 25%-30%. And sometimes in some data sets you might find that even though there are more than 50% missing values in a column, you might have to include that variable in your analysis.
A lot of it depends on the business context as well. In this case we will take this threshold to be 25% and then check the definitions of the columns filtered, to see if there is any column that we might want to keep.

In [ ]:
data.shape[0]

In [ ]:
len(data)

In [ ]:
nulls_percent_df = pd.DataFrame(data.isna().sum()/len(data)).reset_index()
nulls_percent_df.columns = ['column_name', 'nulls_percentage']
nulls_percent_df

Which columns have NA's?

In [ ]:
nulls_percent_df[nulls_percent_df['nulls_percentage']!=0]

In [ ]:
columns_above_threshold = nulls_percent_df[nulls_percent_df['nulls_percentage']>0.25]
columns_above_threshold['column_name']

In [ ]:
drop_columns_list = list(columns_above_threshold['column_name'])
print(drop_columns_list)

From the list above that includes the columns that have over 25% null values, you discussed with your manager you were told that the following columns are important -> **wealth1, wealth2**
We will remove these variables from the above list
RDATE3, RAMNT_3 are important but they have too many null values

# Activity

Create a function that takes a dataframe as an input and a percentual threshold (default value = 0.25) and returns a list of columns with null values greater than the specified threshold

In [ ]:
def remove_cols_with_na(df: pd.DataFrame, thr = 0.25)-> list:
    nulls_percent_df = pd.DataFrame(df.isna().sum()/len(data)).reset_index()
    nulls_percent_df.columns = ['column_name', 'nulls_percentage']
    return list(nulls_percent_df[nulls_percent_df['nulls_percentage'] > thr]['column_name'].values)

In [ ]:
remove_cols_with_na(data, 0.25)

In [ ]:
# End of the activity

# More data cleaning

In [ ]:
data.head()

We can see that there are a lot of columns that have blank spaces which represent no value in this case.
They were not identified as null values by python as they are empty spaces that are read as character values by
python. We will replace those values by NaNs and repeat the analysis

Before we do that we will replace the blank values from the column "MAILCODE" by "A" which would mean the address is okay (pl check the definition of the variable in the description)

In [ ]:
data['MAILCODE'].value_counts()

In [ ]:
data['MAILCODE'].unique()

In [ ]:
data['MAILCODE'] = data['MAILCODE'].apply(lambda x: x.replace(" ", "A"))
#data['MAILCODE'] = np.where(data['MAILCODE'] == " ", "A",data['MAILCODE'] )
#data['MAILCODE'] = data['MAILCODE'].apply(lambda x: x if (x != " ") else "A")
# function(arg1)
#df[col] = list(map(function,df[col]))
# function(arg1, arg2, arg3)
#df[col] = list(map(function,df[col],[arg2]*len(df), [arg3]*len(df)))

In [ ]:
# Now we can replace the rest space characters with np.NaN

In [ ]:
data = data.apply(lambda x: x.replace(" ", np.NaN))

In [ ]:
data.head()

# Activity

Use the previous function to obtain the columns with null values greater than 0.25 and store those columns in a list called "drop_columns_list"

In [ ]:
drop_columns_list = remove_cols_with_na(data)

In [ ]:
# End of activity

We would again repeat the same exercise as the last time. We will discuss it with the team, manager, and/or other stakeholders to see which columns we need to retain here.

Like the last time we will keep the following columns. **wealth1, wealth2 along with these VETERANS, SOLIH**

Remove those column's names from the list of columns to drop.

In [ ]:
# list comprehension
my_list = []

for number in range(1,11):
    my_list.append(number)

my_list

# List comprehension syntax
# [value for item in list]
my_list = [ number for number in range(1,11) ]
my_list

my_list = []

for number in range(1,11):
    if ( number%2 == 0 ):
        my_list.append(number)

my_list

# List comprehension syntax with a conditional
# [value for item in list if (condition)]
my_list = [ number for number in range(1,11) if (number%2 == 0)]
my_list

my_list = []

for number in range(1,11):
    if ( number%2 == 0 ):
        my_list.append(number)
    else:
        my_list.append(0)

my_list

# List comprehension syntax with a conditional with an else
# [value if (condition) else other_value for item in list]
my_list = [ number if (number%2 == 0) else 0 for number in range(1,11) ]
my_list

In [ ]:
def remove_columns(df, thr = 0.25, blacklist=None):
    ...
    if ( blacklist != None):
        drop_columns_list = [ col for col in drop_columns_list if col not in blacklist ]

In [ ]:
drop_columns_list = [ col for col in drop_columns_list if col not in ['WEALTH1','WEALTH2','VETERANS','SOLIH'] ]

In [ ]:
len(drop_columns_list)

# X-y split

Since we have a huge number of features, it would be easier to work independently on numerical features and categorical features.

For the target variables, for now we will retain them both together. But later, we will build a classification model first where we would need the column TARGET_B only.

In [ ]:
Y = data[['TARGET_B', 'TARGET_D']]
Y.head()

In [ ]:
numerical = data.select_dtypes(np.number)
numerical = numerical.drop(columns = ['TARGET_B', 'TARGET_D'])
numerical.head()

In [ ]:
numerical.shape

In [ ]:
categorical = data.select_dtypes([object])
categorical.head()

In [ ]:
categorical.shape

# Working with categorical columns

We will work with the categorical features first. Look at the columns one by one. Some of the operations which we will perform are:

- Replace null values with the most occurring categories
- Reduce the number of categories in a column by grouping

It is important to note that some columns are defined by python as categorical/object types. There might be other columns defined as numerical that we want as categorical. We will look them later when we were working on numerical types.

In [ ]:
categorical.isna().sum()

# Categorical Variables

Here we will try to reduce the number of categories. An ideal way would have been to group the states into
regions. But in this case we will group all the states with counts less than 2500 into one category "other"

In [ ]:
df = pd.DataFrame(categorical['STATE'].value_counts()).reset_index()


df.columns = ['state', 'count']
other_states = list(df[df['count']<2500]['state'])

# other_states = ['CA','AR','MD']
def clean_state(x):
    if x in other_states:
        return 'other'
    else:
        return x

# clean_states(x,other_states) for x, other_states in zip(categorical['STATE'].values,other_states)
categorical['STATE'] = list(map(clean_state, categorical['STATE']))
#categorical['STATE'] = categorical['STATE'].apply(clean_state)

In [ ]:
categorical['STATE'].value_counts()

# 7.01 Lab | Revisiting Machine Learning Case Study

In [ ]:
categorical.isnull().sum()

In [ ]:
drop_list= []
drop_list+= ['OSOURCE', 'ZIP']
display(drop_list)

In [ ]:
cols_null_percentage= categorical.isnull().sum()/len(categorical)
cols_with_high_perc= pd.DataFrame(cols_null_percentage[cols_null_percentage>0.85]).reset_index()
cols_with_high_perc.columns= ['column_name', 'null_percentage']
drop_list.extend(cols_with_high_perc['column_name'])
drop_list

In [ ]:
categorical.drop(columns= drop_list, inplace= True)
categorical.head()

In [ ]:
print(categorical['GENDER'].value_counts(dropna=False))

In [ ]:
categorical['GENDER'] = categorical['GENDER'].fillna('F')
print(categorical['GENDER'].value_counts(dropna=False))

In [ ]:
categorical['GENDER'].replace(['U','J','C','A'], 'other', inplace=True)
categorical['GENDER'].value_counts(dropna=False)

# 7.02


In [ ]:
m = categorical[categorical['DOMAIN'].notna()]['DOMAIN'].mode()[0]

In [ ]:
categorical['DOMAIN'] = categorical['DOMAIN'].fillna(m)

In [ ]:
categorical['DOMAIN_A'] = list(map(lambda x: x[0], categorical['DOMAIN']))
categorical['DOMAIN_B'] = list(map(lambda x: x[1], categorical['DOMAIN']))

In [ ]:
categorical = categorical.drop(columns=['DOMAIN'])

In [ ]:
drop_columns_list += ['RDATE_3', 'RAMNT_3']

In [ ]:
drop_columns_list += ['MDMAUD_R', 'MDMAUD_F','MDMAUD_A']

In [ ]:
m = categorical[categorical['CLUSTER'].notna()]['CLUSTER'].mode()[0]

In [ ]:
categorical['CLUSTER'] = categorical['CLUSTER'].fillna(m)

In [ ]:
m = categorical[categorical['HOMEOWNR'].notna()]['HOMEOWNR'].mode()[0]

In [ ]:
categorical['HOMEOWNR'] = categorical['HOMEOWNR'].fillna(m)

In [ ]:
drop_columns_list += [col for col in categorical.columns if (("RFA" in col) and ( col not in ['RFA_2R','RFA_2A']))]

In [ ]:
cat_columns_to_drop = [col for col in drop_columns_list if col in list(categorical.columns)]


In [ ]:
categorical = categorical.drop(columns=cat_columns_to_drop, axis=1)


In [ ]:
categorical.columns

In [ ]:
remove_cols_with_na(numerical,0.25)

In [ ]:
numerical['AGE'] = numerical["AGE"].fillna(np.median(numerical['AGE']))

In [ ]:
m = numerical['INCOME'].mode()[0]
numerical['INCOME'] = numerical['INCOME'].fillna(m) # Replacing the null values with the most represented categoty

In [ ]:
numerical['CLUSTER2'] = numerical['CLUSTER2'].fillna(np.ceil(np.mean(numerical['CLUSTER2'])))

In [ ]:
num_cols_to_drop = [col for col in list(numerical.columns) if "ADATE_" in col]
numerical.drop(columns=num_cols_to_drop, axis = 1, inplace = True)

# Lab | Feature engineering

In [ ]:
null_count= numerical.isna().sum()

In [ ]:
# Cleaning the GEOCODE2 column
categorical['GEOCODE2'].value_counts(dropna=False)

In [ ]:
m = categorical['GEOCODE2'].mode()[0]
categorical['GEOCODE2'] = categorical['GEOCODE2'].fillna(m) 
categorical['GEOCODE2'].value_counts(dropna=False)

In [ ]:
# Cleaning the WEALTH1 column
numerical['WEALTH1'].value_counts(dropna=False)

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(16,5))
sns.histplot(numerical['WEALTH1'], ax = ax[0])
sns.histplot(numerical["WEALTH1"].fillna(np.mean(numerical[numerical['WEALTH1'].notna()]['WEALTH1'])), ax = ax[1])
sns.histplot(numerical['WEALTH1'].fillna(np.median(numerical[numerical['WEALTH1'].notna()]['WEALTH1'])),ax = ax[2])
plt.show()


In [ ]:
median= np.median(numerical['WEALTH1'].dropna())
numerical['WEALTH1'] = numerical['WEALTH1'].fillna(median)
numerical['WEALTH1'].value_counts(dropna=False)

In [ ]:
print(numerical['ADI'].value_counts())
print(numerical['ADI'].isna().sum()) #only 132 NaN values
ADI_mean= np.mean(numerical[numerical['ADI'].notna()]['ADI'])
ADI_median= np.median(numerical[numerical['ADI'].notna()]['ADI'])
print("mean:", ADI_mean)
print("median:", ADI_median)

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(16,5))
sns.histplot(numerical['ADI'], ax = ax[0])
sns.histplot(numerical["ADI"].fillna(np.mean(numerical[numerical['ADI'].notna()]['ADI'])), ax = ax[1])
sns.histplot(numerical['ADI'].fillna(np.median(numerical[numerical['ADI'].notna()]['ADI'])),ax = ax[2])
plt.show()
# I do not see a siginificant difference, and as the value are integers, i will replace with the median value
numerical['ADI'] = numerical['ADI'].fillna(ADI_median)

In [ ]:
print(numerical['ADI'].isna().sum())

In [ ]:
print(numerical['DMA'].value_counts(dropna=False))
print(numerical['DMA'].isna().sum()) #only 132 NaN values
DMA_mean= np.mean(numerical[numerical['DMA'].notna()]['DMA'])
DMA_median= np.median(numerical[numerical['DMA'].notna()]['DMA'])
print("mean:", DMA_mean)
print("median:", DMA_median)

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(16,5))
sns.histplot(numerical['DMA'], ax = ax[0])
sns.histplot(numerical["DMA"].fillna(DMA_mean), ax = ax[1])
sns.histplot(numerical['DMA'].fillna(DMA_median),ax = ax[2])
plt.show()
# I do not see a siginificant difference, and as the value are integers, i will replace with the median value
numerical['DMA'] = numerical['DMA'].fillna(DMA_median)

In [ ]:
print(numerical['DMA'].isna().sum())

In [ ]:
print(numerical['MSA'].value_counts(dropna=False))
print(numerical['MSA'].isna().sum()) #only 132 NaN values
MSA_mean= np.mean(numerical[numerical['MSA'].notna()]['MSA'])
MSA_median= np.median(numerical[numerical['MSA'].notna()]['MSA'])
print("mean:", MSA_mean)
print("median:", MSA_median)

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(16,5))
sns.histplot(numerical['MSA'], ax = ax[0])
sns.histplot(numerical["MSA"].fillna(MSA_mean), ax = ax[1])
sns.histplot(numerical['MSA'].fillna(MSA_median),ax = ax[2])
plt.show()
# I do not see a siginificant difference, and as the value are integers, i will replace with the median value
numerical['MSA'] = numerical['MSA'].fillna(MSA_median)

In [ ]:
print(numerical['MSA'].isna().sum())